---
#### 【得点】
#### (v6) 0.74641・・・ベースライン
#### (v6) 0.77751・・・Ticketの数字列追加
#### Age・・・平均
#### Fare, Embarked・・・最頻値
#### Cabin,Ticket, Name・・・抹消
#### model・・・LGBM デフォルト
---



---
#### (v7)
#### Family = SibSp + Parch
#### Age・・・平均
#### Fare, Embarked・・・最頻値
#### Cabin,Ticket, Name・・・抹消
#### model・・・LGBM デフォルト
#### (v11)
#### Tnum列追加
#### 記号は傾向見えたけど後回しにする
---


In [1]:
# ライブラリのインポート
import numpy as np
import pandas as pd

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

/kaggle/input/titanic/train.csv
/kaggle/input/titanic/test.csv
/kaggle/input/titanic/gender_submission.csv


In [2]:
# 訓練テータ読み込み
train_df = pd.read_csv("/kaggle/input/titanic/train.csv")
test_df = pd.read_csv("/kaggle/input/titanic/test.csv")
submission = pd.read_csv("/kaggle/input/titanic/gender_submission.csv")

In [3]:
# 乱数を固定する
import random
np.random.seed(1234)
random.seed(1234)

In [4]:
# データを可視化する
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns
plt.style.use("ggplot")

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


#### 考えつくこと
- SibSpとParchで変数追加(Familyとする)
- Fareの欠損値埋めにPclassとFareを利用
- Ageの補完方法
 - 年齢との相関があるパラメータを探す(Pclass,Fare,Family)
- Nameは敬称

In [5]:
# 全データ結合
all_df = pd.concat([train_df,test_df],sort=False).reset_index(drop=True)

#### Ticketの処理
---
#### Ticketの番号の処理
---

In [6]:
# # Ticketと生存率を比較したい。まずは数字で分ける
# # 正規表現
# import re

# # スペース分割したときに要素が1つ(数字のみ)か2つ(文字含む)か
# p = re.compile('[^\s]+')
# Tnum = []

# # 末尾要素を抽出
# for i in range(len(all_df)):
#     Tnum.append(p.findall(all_df["Ticket"][i])[-1])
        
# Tnum_df = pd.DataFrame(Tnum)
# all_df["Tnum"] = Tnum_df

# # Ticketに"LINE"という要素があるので、該当の行を落とす
# index = all_df.index[all_df["Tnum"] == "LINE"]
# all_df = all_df.drop(index).reset_index()

# # floatからint64にキャスト
# all_df["Tnum"] = all_df["Tnum"].astype("int64")

# # Numと生存率を比較する
# # "Num", max=310x104, 50%=11.1x10^4, min=2
# all_df["Tnum"].describe()

# # 数字を「Low/Mid/High」に分ける
# div_df = pd.DataFrame([])

# # インデックスを3等分する
# row_25 = 13600
# row_50 = 111000
# row_75 = 347000
# row_100 = 3101317

# # 数字を3段階に分ける
# div_df["25%"] = all_df[["Survived","Tnum"]].loc[all_df["Tnum"]<row_25].dropna()\
#                     .groupby("Survived").count()

# div_df["50%"] = all_df[["Survived","Tnum"]].loc[(all_df["Tnum"]>=row_25) & (all_df["Tnum"]<row_50)].dropna()\
#                     .groupby("Survived").count()

# div_df["75%"] = all_df[["Survived","Tnum"]].loc[(all_df["Tnum"]>=row_50) & (all_df["Tnum"]<row_75)].dropna()\
#                     .groupby("Survived").count()

# div_df["100%"] = all_df[["Survived","Tnum"]].loc[(all_df["Tnum"]>=row_75)].dropna()\
#                     .groupby("Survived").count()

# # 生存率を追加する
# Suv_25 = div_df.loc[1,"25%"]/(div_df.loc[1,"25%"] + div_df.loc[0,"25%"])
# Suv_50 = div_df.loc[1,"50%"]/(div_df.loc[1,"50%"] + div_df.loc[0,"50%"])
# Suv_75 = div_df.loc[1,"75%"]/(div_df.loc[1,"75%"] + div_df.loc[0,"75%"])
# Suv_100 = div_df.loc[1,"100%"]/(div_df.loc[1,"100%"] + div_df.loc[0,"100%"])
# print(Suv_25,Suv_50,Suv_75,Suv_100,sep="\n")

#### Ticketの記号の処理
---

In [7]:
# # Ticketと生存率を比較したい。まずは数字で分ける
# # 正規表現
# import re

# # スペース分割したときに要素が1つ(数字のみ)か2つ(文字含む)か
# p = re.compile('[^\s]+')
# Tsign = []

# # 記号を持つ要素のみ抽出。それ以外はNaN
# for i in range(len(all_df)):
#     if(len(p.findall(all_df["Ticket"][i]))==2):
#         Tsign.append(p.findall(all_df["Ticket"][i][0]))
#     else:
#         Tsign.append(["missing"])
    
# # データフレーム化
# Tsign_df = pd.DataFrame(Tsign)
# all_df["Tsign"] = Tsign_df

# # P:98,S:83,C:77,A:41,W:19,F:13
# all_df["Tsign"].value_counts()

# # チケット記号ごとの生存率比較
# diff = pd.DataFrame([])

# diff["P"] = all_df.loc[all_df["Tsign"]=="P",["Survived","Tsign"]].groupby("Survived").count()
# diff["S"] = all_df.loc[all_df["Tsign"]=="S",["Survived","Tsign"]].groupby("Survived").count()
# diff["C"] = all_df.loc[all_df["Tsign"]=="C",["Survived","Tsign"]].groupby("Survived").count()
# diff["A"] = all_df.loc[all_df["Tsign"]=="A",["Survived","Tsign"]].groupby("Survived").count()
# diff["W"] = all_df.loc[all_df["Tsign"]=="W",["Survived","Tsign"]].groupby("Survived").count()
# diff["F"] = all_df.loc[all_df["Tsign"]=="F",["Survived","Tsign"]].groupby("Survived").count()

# # 記号別の生存率
# all_df.loc[all_df["Tsign"]=="P",["Survived","Embarked","Pclass","PassengerId"]].groupby(["Survived","Embarked", "Pclass"]).count()

In [8]:
# 欠損値
all_df.isnull().sum()

PassengerId       0
Survived        418
Pclass            0
Name              0
Sex               0
Age             263
SibSp             0
Parch             0
Ticket            0
Fare              1
Cabin          1014
Embarked          2
dtype: int64

#### Pclassの比較
---

In [9]:
# # 家族の数、部屋のグレード、性別、年齢
# # tmp = pd.DataFrame([])
# cls = all_df[["Survived","Pclass","PassengerId"]].dropna().groupby(["Survived","Pclass"]).count().unstack()

# # Pclassの生存率比較
# # (1)0.63, (2)0.47, (3)0.24
# cls2 = []
# cls2.append(tmp.iloc[1,0]/(tmp.iloc[0,0]+tmp.iloc[1,0]))
# cls2.append(tmp.iloc[1,1]/(tmp.iloc[0,1]+tmp.iloc[1,1]))
# cls2.append(tmp.iloc[1,2]/(tmp.iloc[0,2]+tmp.iloc[1,2]))

In [10]:
# # Pclass高い方が年齢上
# # (1)39.2, (2)29.5, (3)24.8
# all_df[["Pclass","Age"]].dropna().groupby("Pclass").mean()

# # そのときの性別は？
# sxn = all_df[["Pclass","Sex","PassengerId"]].dropna().groupby(["Pclass","Sex"]).count().unstack()

# # 男性比率
# # (1)0.55,(2)0.61,(3)0.70
# sxn2 = []
# sxn2.append(sxn.iloc[0,1]/(sxn.iloc[0,0]+sxn.iloc[0,1]))
# sxn2.append(sxn.iloc[1,1]/(sxn.iloc[1,0]+sxn.iloc[1,1]))
# sxn2.append(sxn.iloc[2,1]/(sxn.iloc[2,0]+sxn.iloc[2,1]))

# # Pclassごとの男女別年齢
# # (1)f37.0,m41.0,(2)f27.5,m30.8,(3)f22.2,m26.0
# all_df[["Pclass","Age","Sex"]].dropna().groupby(["Pclass","Sex"]).mean()

#### Ageの解析
#### PclassでSex別の年齢がくっきり分かれるので適用する
---

In [11]:
# # 年齢ごとのインデックスを抽出
# data0s = all_df.loc[all_df.index[all_df["Age"]<10]].reset_index()
# data10s = alldf.loc[all_df.index[(all_df["Age"]<20) & (all_df["Age"]>=10)]].reset_index()
# data20s = all_df.index[(all_df["Age"]<30) & (all_df["Age"]>=20)]
# # ind30s = all_df.index[(all_df["Age"]<40) & (all_df["Age"]>=30)]
# # ind40s = all_df.index[(all_df["Age"]<50) & (all_df["Age"]>=40)]
# # ind50s = all_df.index[(all_df["Age"]<60) & (all_df["Age"]>=50)]
# # ind60s = all_df.index[(all_df["Age"]<70) & (all_df["Age"]>=60)]
# # ind70s = all_df.index[all_df["Age"]>=70]

# # all_df[["Survived","Pclass","PassengerId","Age"]].dropna().groupby(["Pclass","Survived"]).count().unstack()

In [12]:
# 年齢ごとのインデックスを抽出
# ind0s = all_df.index[all_df["Age"]<10]
# ind10s = all_df.index[(all_df["Age"]<20) & (all_df["Age"]>=10)]
# ind20s = all_df.index[(all_df["Age"]<30) & (all_df["Age"]>=20)]
# ind30s = all_df.index[(all_df["Age"]<40) & (all_df["Age"]>=30)]
# ind40s = all_df.index[(all_df["Age"]<50) & (all_df["Age"]>=40)]
# ind50s = all_df.index[(all_df["Age"]<60) & (all_df["Age"]>=50)]
# ind60s = all_df.index[(all_df["Age"]<70) & (all_df["Age"]>=60)]
# ind70s = all_df.index[all_df["Age"]>=70]

In [13]:
#all_df[["Survived","Pclass","Sex","PassengerId"]].loc[ind0s].dropna().groupby(["Survived","Pclass","Sex"]).count().unstack()

In [14]:
# 年齢ごとの生存率を比較
# difAge = pd.DataFrame([])

# difAge["0s"] = all_df.loc[ind0s,["Survived","Age"]].groupby("Survived").count()
# difAge["10s"] = all_df.loc[ind10s,["Survived","Age"]].groupby("Survived").count()
# difAge["20s"] = all_df.loc[ind20s,["Survived","Age"]].groupby("Survived").count()
# difAge["30s"] = all_df.loc[ind30s,["Survived","Age"]].groupby("Survived").count()
# difAge["40s"] = all_df.loc[ind40s,["Survived","Age"]].groupby("Survived").count()
# difAge["50s"] = all_df.loc[ind50s,["Survived","Age"]].groupby("Survived").count()
# difAge["60s"] = all_df.loc[ind60s,["Survived","Age"]].groupby("Survived").count()
# difAge["70s"] = all_df.loc[ind70s,["Survived","Age"]].groupby("Survived").count()

In [15]:
# SuvAge = []
# SuvAge.append(difAge.loc[1,"0s"]/(difAge.loc[1,"0s"] + difAge.loc[0,"0s"]))
# SuvAge.append(difAge.loc[1,"10s"]/(difAge.loc[1,"10s"] + difAge.loc[0,"10s"]))
# SuvAge.append(difAge.loc[1,"20s"]/(difAge.loc[1,"20s"] + difAge.loc[0,"20s"]))
# SuvAge.append(difAge.loc[1,"30s"]/(difAge.loc[1,"30s"] + difAge.loc[0,"30s"]))
# SuvAge.append(difAge.loc[1,"40s"]/(difAge.loc[1,"40s"] + difAge.loc[0,"40s"]))
# SuvAge.append(difAge.loc[1,"50s"]/(difAge.loc[1,"50s"] + difAge.loc[0,"50s"]))
# SuvAge.append(difAge.loc[1,"60s"]/(difAge.loc[1,"60s"] + difAge.loc[0,"60s"]))
# SuvAge.append(difAge.loc[1,"70s"]/(difAge.loc[1,"70s"] + difAge.loc[0,"70s"]))

In [16]:
# columns1 = ["0s","10s","20s","30s","40s","50s","60s","70s"]
# pd.DataFrame(data=SuvAge)

In [17]:
# Pclassの人数
# 1(323人)、2(277人)、3(709人)

# 平均値
# 年齢: 1(39.2歳)、2(29.5歳)、3(24.8歳)
# 家族数: 1(0.80)、2(0.76)、3(0.97)
# 代金: 1(87.5)、2(21.2)、3(13.3)

# 性別: 1(男:179人、女:144人)、2(男:171人、女:106人)、3(男:493人、女:216人)



# all_df[["Pclass","Family","PassengerId"]].dropna().groupby(["Pclass","Family"]).count().unstack()
# all_df[["Pclass","Sex","PassengerId"]].dropna().groupby(["Pclass","Sex"]).count().unstack()
# all_df[["Pclass","PassengerId"]].groupby("Pclass").count().plot.bar()

In [18]:
# all_df["Age"].mean()

#### ここから説明変数の処理
---

In [19]:
# Ticketと生存率を比較したい。まずは数字で分ける
# 正規表現
import re

# スペース分割したときに要素が1つ(数字のみ)か2つ(文字含む)か
p = re.compile('[^\s]+')
Tnum = []

# 末尾要素を抽出
for i in range(len(all_df)):
    Tnum.append(p.findall(all_df["Ticket"][i])[-1])
        
Tnum_df = pd.DataFrame(Tnum)
all_df["Tnum"] = Tnum_df

# Ticketに"LINE"という要素があるので、該当の行を落とす
index = all_df.index[all_df["Tnum"] == "LINE"]
all_df = all_df.drop(index).reset_index()

# floatからint64にキャスト
all_df["Tnum"] = all_df["Tnum"].astype("int64")

In [20]:
# SibSpとParchを統合
# 正解率は変化なかったので実施
all_df["Family"] = all_df["SibSp"] + all_df["Parch"]
all_df.drop(columns=["SibSp","Parch"],inplace=True)

In [21]:
# Pclassごとの男女別年齢
# (1)f37.0,m41.0,(2)f27.5,m30.8,(3)f22.2,m26.0
# clsAge = all_df[["Pclass","Age","Sex"]].dropna().groupby(["Pclass","Sex"]).mean().unstack()

# # Pclassごとの男女別年齢を記録
# cls1f = clsAge.iloc[0,0]
# cls1m = clsAge.iloc[0,1]
# cls2f = clsAge.iloc[1,0]
# cls2m = clsAge.iloc[1,1]
# cls3f = clsAge.iloc[2,0]
# cls3m = clsAge.iloc[2,1]

# # Pclass=1の女性
# all_df.loc[(all_df["Pclass"]==1) & (all_df["Sex"]=="female"),"Age"].fillna(37.0,inplace=True)
# all_df.loc[(all_df["Pclass"]==1) & (all_df["Sex"]=="male"),"Age"].fillna(41.0,inplace=True)
# all_df.loc[(all_df["Pclass"]==2) & (all_df["Sex"]=="female"),"Age"].fillna(27.5,inplace=True)
# all_df.loc[(all_df["Pclass"]==2) & (all_df["Sex"]=="male"),"Age"].fillna(30.8,inplace=True)
# all_df.loc[(all_df["Pclass"]==3) & (all_df["Sex"]=="female"),"Age"].fillna(22.2,inplace=True)
# all_df.loc[(all_df["Pclass"]==3) & (all_df["Sex"]=="male"),"Age"].fillna(26.0,inplace=True)

# 平均値で補完(Age)
all_df["Age"].fillna(all_df["Age"].mean(), inplace=True)

# 最頻値で補完
# (Fareは1人、Embarkedは2人しか欠損無いため考慮不要)
all_df["Fare"].fillna(all_df["Fare"].median(), inplace=True)
all_df["Embarked"].fillna("S", inplace=True)

# 列を落とす(Cabin, Name, Ticket)
all_df.drop(columns="Cabin", inplace=True)
all_df.drop(columns="Name", inplace=True)
all_df.drop(columns="Ticket", inplace=True)

In [22]:
# カテゴリ変数は、Sex, Embarked, Cabin, Ticket, Name
# ラベルエンコード
from sklearn.preprocessing import LabelEncoder

# カテゴリ変数
categories = all_df.columns[all_df.dtypes==object]
print(categories)

for cat in categories:
    le = LabelEncoder()
    print(cat)
    le = le.fit(all_df[cat])
    all_df[cat] = le.transform(all_df[cat])

Index(['Sex', 'Embarked'], dtype='object')
Sex
Embarked


In [23]:
all_df.isnull().sum()

index            0
PassengerId      0
Survived       418
Pclass           0
Sex              0
Age              0
Fare             0
Embarked         0
Tnum             0
Family           0
dtype: int64

In [24]:
# 訓練データに分割
train_X = all_df[~all_df["Survived"].isnull()].drop("Survived",axis=1).reset_index(drop=True)
train_Y = all_df[~all_df["Survived"].isnull()]["Survived"].reset_index(drop=True)

# テストデータ
test_X = all_df[all_df["Survived"].isnull()].drop("Survived",axis=1).reset_index(drop=True)
test_Y = submission

In [25]:
# import lightgbm as lgb
import optuna.integration.lightgbm as lgb
from sklearn.model_selection import train_test_split

# 分割する
X_train, X_val, Y_train, Y_val = train_test_split(train_X, train_Y, test_size=0.2)

In [26]:
# パラメータ指定
lgbm_params = {
    "objective": "binary",# 2値分類
    "metric": "binary_logloss",
    "num_iterations": 1000,# 学習のイテレーション回数
    "num_leaves": 63,# 分岐の終着点の数
    "max_depth": 7,# 木の最大深さ
    "min_data_in_leaf": 20,# 葉の行き着くデータ
    "learning_rate": 0.01,# 学習率
    "bagging_fraction": 0.8,
    "bagging_freq": 1,
    "feature_fraction": 1.0,
    'feature_pre_filter': False,
    'lambda_l1': 0.0053067687634611995,
    'lambda_l2': 3.803611564502165e-05,
    'min_child_samples': 20,
    "random_seed": 1234
}

In [27]:
# import optuna
# from sklearn.metrics import log_loss

# def objective(trial):
#     # パラメータ指定
#     lgbm_params = {
#         "objective": "binary",# 2値分類
#         "random_seed": 1234,# ランダムシード
#         "learning_rate": 0.05,# 学習率
#         "n_estimators": 100,# 最大学習サイクル数 (100)

#         "num_leaves": trial.suggest_int("num_leaves", 4, 64),# 分岐の終着点の数
#         "max_bin": trial.suggest_int("max_bin", 50, 200),# 特徴量の分け方
#         "bagging_fraction": trial.suggest_uniform("bagging_fraction", 0.4, 0.9),#サンプリング
#         "bagging_freq": trial.suggest_int("bagging_freq", 1, 10),# Baggingを何回に1回行うか
#         "feature_fraction": trial.suggest_uniform("feature_fraction", 0.4, 0.9),
#         "min_data_in_leaf": trial.suggest_int("min_data_in_leaf", 2, 16),# 葉の行き着くデータ
#         "min_sum_hessian_in_leaf": trial.suggest_int("min_sum_hessian_in_leaf", 1, 10),#一つの葉のデータの最小個数
#     }
    
#     # データセットの準備
#     lgb_train = lgb.Dataset(X_train, Y_train)
#     lgb_val = lgb.Dataset(X_val, Y_val,reference=lgb_train)

#     # モデル実行
#     model = lgb.train(params = lgbm_params,#
#                   train_set = lgb_train,
#                   valid_sets = lgb_val,
#                   num_boost_round = 100,# 
#                   early_stopping_rounds = 20,
#                   verbose_eval = -1,
#                  )
    
#     y_pred = model.predict(X_val, num_iteration=model.best_iteration)
#     score = np.sqrt(log_loss(Y_val, y_pred))
    
#     return score

In [28]:
# study = optuna.create_study(sampler=optuna.samplers.RandomSampler(seed=0))
# study.optimize(objective, n_trials=50)
# study.best_params

In [29]:
# model.params

In [30]:
# {'objective': 'binary',
#  'metric': 'binary_logloss',
#  'num_leaves': 63,
#  'max_depth': 7,
#  'min_data_in_leaf': 20,
#  'learning_rate': 0.01,
#  'bagging_fraction': 0.8,
#  'bagging_freq': 1,
#  'feature_fraction': 1.0,
#  'random_seed': 1234,
#  'feature_pre_filter': False,
#  'lambda_l1': 0.0053067687634611995,
#  'lambda_l2': 3.803611564502165e-05,
#  'min_child_samples': 20,
#  'num_iterations': 1000,
#  'early_stopping_round': None}
# データセットの準備
lgb_train = lgb.Dataset(X_train, Y_train)
lgb_val = lgb.Dataset(X_val, Y_val,reference=lgb_train)

# モデル実行
model = lgb.train(params = lgbm_params,
                  train_set = lgb_train,
                  valid_sets = lgb_val,
                  num_boost_round = 100,
                  early_stopping_rounds = 20,
                 )

[I 2023-11-15 21:51:53,626] A new study created in memory with name: no-name-1cca5d98-0249-4650-8f00-0bb5d7b31979
feature_fraction, val_score: inf:   0%|          | 0/7 [00:00<?, ?it/s]/opt/conda/lib/python3.10/site-packages/lightgbm/engine.py:177: UserWarning: Found `num_iterations` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")


[LightGBM] [Warning] min_data_in_leaf is set=20, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=20
[LightGBM] [Warning] min_data_in_leaf is set=20, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=20
[LightGBM] [Info] Number of positive: 270, number of negative: 439
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000140 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 908
[LightGBM] [Info] Number of data points in the train set: 709, number of used features: 9
[LightGBM] [Warning] min_data_in_leaf is set=20, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=20
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.380818 -> initscore=-0.486077
[LightGBM] [Info] Start training from score -0.486077
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[1]	va

feature_fraction, val_score: 0.414657:  14%|#4        | 1/7 [00:00<00:01,  3.44it/s]

[317]	valid_0's binary_logloss: 0.421779
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[318]	valid_0's binary_logloss: 0.421622
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[319]	valid_0's binary_logloss: 0.421577
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[320]	valid_0's binary_logloss: 0.421729
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[321]	valid_0's binary_logloss: 0.421861
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[322]	valid_0's binary_logloss: 0.421658
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[323]	valid_0's binary_logloss: 0.421705
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[324]	valid_0's binary_logloss: 0.42166
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[325]	valid_0's binary_logloss: 0.421583
[LightGBM] [Warning] No further 

/opt/conda/lib/python3.10/site-packages/lightgbm/engine.py:177: UserWarning: Found `num_iterations` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")
feature_fraction, val_score: 0.414657:  29%|##8       | 2/7 [00:00<00:01,  3.65it/s]

...

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[234]	valid_0's binary_logloss: 0.409754
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[235]	valid_0's binary_logloss: 0.409375
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[236]	valid_0's binary_logloss: 0.409229
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[237]	valid_0's binary_logloss: 0.409089
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[238]	valid_0's binary_logloss: 0.409037
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[239]	valid_0's binary_logloss: 0.408988
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[240]	valid_0's binary_logloss: 0.408944
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[241]	valid_0's binary_logloss: 0.408652
[LightGBM] [Warning] No further splits with positive gain, best gain: -i

/opt/conda/lib/python3.10/site-packages/lightgbm/engine.py:177: UserWarning: Found `num_iterations` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")
min_data_in_leaf, val_score: 0.400564:  80%|########  | 4/5 [00:00<00:00,  4.37it/s]

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[188]	valid_0's binary_logloss: 0.421993
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[189]	valid_0's binary_logloss: 0.421573
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[190]	valid_0's binary_logloss: 0.421418
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[191]	valid_0's binary_logloss: 0.421405
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[192]	valid_0's binary_logloss: 0.421445
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[193]	valid_0's binary_logloss: 0.420789
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[194]	valid_0's binary_logloss: 0.420143
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[195]	valid_0's binary_logloss: 0.419507
[LightGBM] [Warning] No further splits with positive gain, best gain: -i

/opt/conda/lib/python3.10/site-packages/lightgbm/engine.py:177: UserWarning: Found `num_iterations` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")
min_data_in_leaf, val_score: 0.400564: 100%|##########| 5/5 [00:01<00:00,  4.38it/s]

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[153]	valid_0's binary_logloss: 0.436057
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[154]	valid_0's binary_logloss: 0.43539
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[155]	valid_0's binary_logloss: 0.434783
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[156]	valid_0's binary_logloss: 0.434144
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[157]	valid_0's binary_logloss: 0.43396
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[158]	valid_0's binary_logloss: 0.433737
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[159]	valid_0's binary_logloss: 0.433572
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[160]	valid_0's binary_logloss: 0.433143
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf

In [31]:
# 正解率を出す
from sklearn.metrics import accuracy_score
# accuracy_score()
acc = model.predict(X_val, num_iteration=model.best_iteration)
accuracy_score(Y_val, np.round(acc))

0.8426966292134831

In [32]:
# テストデータに対して予測する
tmp = np.round(model.predict(test_X, num_iteration=model.best_iteration))
tmp = np.array(tmp, dtype=np.int64)

In [33]:
# テストデータに対して予測する
submission["Survived"] = tmp

In [34]:
submission.to_csv("submission.csv",index=False)

In [35]:
submission

,PassengerId,Survived
0,892,0
1,893,0
2,894,0
3,895,0
4,896,1
...,...,...
413,1305,0
414,1306,1
415,1307,0
416,1308,0
